# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [4]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import pandas as pd
import requests
import os

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [5]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df=pd.read_csv(os.path.join("..","Vacation_Search","WeatherPy_vacation.csv"))


# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,0,Bethel,US,49.50,41.3712,-73.4140,Hampton Inn Danbury
1,11,Longyearbyen,SJ,35.44,78.2186,15.6401,The Vault
2,13,Albany,US,40.59,42.6001,-73.9662,No hotel found
3,16,Bose,IT,43.29,45.4612,8.0137,Hotel Ristorante Taverna Verde
4,23,Torbay,CA,24.28,47.6666,-52.7314,No hotel found


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [6]:
# Configure the map plot
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Weather Description"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [7]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df['City']=='Maryville']
vacation_end = vacation_df.loc[vacation_df['City']=='Maryville']
vacation_stop1 = vacation_df.loc[vacation_df['City']=='Hamilton']
vacation_stop2 = vacation_df.loc[vacation_df['City']=='Woodhaven'] 
vacation_stop3 = vacation_df.loc[vacation_df['City']=='North Platte'] 

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [8]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,124,Maryville,US,41.25,35.7565,-83.9705,Main lodge
1,276,Hamilton,US,32.81,39.1834,-84.5333,North Vista Manor
2,670,Woodhaven,US,32.09,42.1389,-83.2416,Best Western
3,211,North Platte,US,26.08,41.1239,-100.7654,Ramada


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [10]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['Lat', 'Lng']].copy()

# Display sample data
waypoints_df

,Lat,Lng
0,35.7565,-83.9705
1,39.1834,-84.5333
2,42.1389,-83.2416
3,41.1239,-100.7654


## Use GeoViews to create map that shows the four cities in the itinerary

In [25]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country", "Weather Description"]
)

In [26]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [27]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [28]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'e4e9f3df8f5846c79e93b8b750ff438a',
 'waypoints': '35.7565,-83.9705|39.1834,-84.5333|42.1389,-83.2416|41.1239,-100.7654'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [30]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params).json()

route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [-83.9705, 35.7565], 'original_index': 0},
     {'location': [-84.5333, 39.1834], 'original_index': 1},
     {'location': [-83.2416, 42.1389], 'original_index': 2},
     {'location': [-100.7654, 41.1239], 'original_index': 3}],
    'units': 'metric',
    'distance': 2440188,
    'distance_units': 'meters',
    'time': 80146.713,
    'legs': [{'distance': 441818,
      'time': 15080.899,
      'steps': [{'from_index': 0,
        'to_index': 2,
        'distance': 78,
        'time': 7.085,
        'instruction': {'text': 'Drive southwest on East Broadway Avenue/US 411.'}},
       {'from_index': 2,
        'to_index': 55,
        'distance': 2443,
        'time': 182.285,
        'instruction': {'text': 'Turn right onto North Cusick Street.'}},
       {'from_index': 55,
        'to_index': 219,
        'distance': 13030,
        'time': 530.145,
        'instruction': {'text': 'Turn right 

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [31]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response["features"][0]["geometry"]["coordinates"]

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [32]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for coordinate in leg:
        longitude.append(coordinate[0])
        latitude.append(coordinate[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [35]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df['Longitude'] = longitude
route_df['Latitude'] = latitude

# Display sample data
route_df.head()

,Longitude,Latitude
0,-83.970641,35.756596
1,-83.970673,35.756565
2,-83.971176,35.756040
3,-83.971766,35.756438
4,-83.972280,35.756803


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [40]:
# Configure the route path by using the GeoViews' Path function
route_path = route_path = gv.Path(
    route_df,
).opts(
    height=500,
    width=700,
    line_width = 3,
    color = "blue"
)

In [42]:
# Display a composed plot by using the route_map and route_path objects
waypoints_map * route_path

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)
   .Path.I   :Path   [Longitude,Latitude]